# Распознавание цифр

В этот раз мы будем использовать датасет MNIST, который содержить около 60 000 картинок с цифрами, которые были написаны отруки. Каждая картинка в нём имеет размер 28х28 пикселей. Цифры: от 0 до 9. 

То есть в этом случае мы имеем дело с мультиклассовой классификацией. 

> MNIST расшифровывается как Modified National Institute of Standart and Technology. 

Сначала импортируем все необходимые библиотеки. 

> Для распознавания будем использовать torch 

In [ ]:
import torch
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
from torchvision.transforms import transforms
import torch.nn.functional as F

Инициализируем параметры 

In [ ]:
CONFIG = { 
    "input_size": 784,      # 28x28 
    "hidden_size_1": 200,   # размер 1-го скрытого слоя
    "hidden_size_2": 150,   # размер 2-го скрытого слоя
    "hidden_size_3": 100,   # размер 3-го скрытого слоя
    "hidden_size_4": 80,    # размер 4-го скрытого слоя 
    "output": 10,           # кол-во выходов сети (т.к. цифры от 0 до 9)
    "bach_size": 100,
    "lr_rate": 0.01
}

Загружаем данные, повезло, что в библиотеке torchvision уже есть функция, которая всё сделает за нас. 

In [ ]:
# загружаем обучающую выборку 
train_data = torchvision.datasets.MNIST(
    "mnist_content", train=True, transform=transforms.ToTensor(), download=True
)

In [ ]:
# загружаем тестовую выборку
test_data = torchvision.datasets.MNIST(
    "mnist_content", train=False, transform=transforms.ToTensor(), download=False
)

Создаём лоядеры данных. Эти лоадеры прослойкой между выборками и кодом модели, так как модель ожидает данные в определённой форме, лоадер делает эту "грязную" работу за нас. Что есть удобство! 

In [ ]:
train_dataloader=torch.utils.data.DataLoader(dataset=train_data, 
                                  batch_size=CONFIG["bach_size"],shuffle=True)
test_dataloader=torch.utils.data.DataLoader(dataset=test_data, 
                                  batch_size=CONFIG["bach_size"],shuffle=True)

In [ ]:
data=iter(train_dataloader)
samples,labels=next(data)
print(f"number of samples{samples.shape}")
print(f"number of labels {labels.shape}")

Давайте посмотрим на картинки 

In [ ]:
plt.figure(figsize=(10,5))
for i in range(10):
    plt.subplot(2,5,i+1)
    plt.imshow(samples[i][0],cmap='BuPu')
plt.show()

Очевидно, что все люди пишут цифры по-разному, и временами даже сам человек не может быть до конца уверен, что за цифра на картинке. 

Давайте напишем класс самой модели, которая будет обучаться. 

In [ ]:
class MNIST(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, hidden_size, output):
        super().__init__()
        self.f_connected1=nn.Linear(input_size,hidden_size1)
        self.f_connected2=nn.Linear(hidden_size1,hidden_size2)
        self.f_connected3=nn.Linear(hidden_size2,hidden_size3)
        self.f_connected4=nn.Linear(hidden_size3,hidden_size)
        self.out_connected=nn.Linear(hidden_size,output)

    def forward(self,x):
            out=F.relu(self.f_connected1(x)) 
            out=F.relu(self.f_connected2(out))
            out=F.relu(self.f_connected3(out))
            out=F.relu(self.f_connected4(out))
            out=self.out_connected(out)
            return out

* `nn.Module` - это класс из pytorch, его можно рассматривать как довольно "удобного" родителя для своих моделей. 
* `nn.Linear` - это линейный слой
*  `F.relu` - функция активации relu (вообще внутри torch есть много разных уже реализованных функций активации: relu, leaky relu, softmax, sigmoid etc.)

Давайте создадим объект нашей модели, а параметры для неё возьмём из конфига. 

In [ ]:
model = MNIST(
    input_size=CONFIG["input_size"], 
    hidden_size1=CONFIG["hidden_size_1"],
    hidden_size2=CONFIG["hidden_size_2"], 
    hidden_size3=CONFIG["hidden_size_3"], 
    hidden_size=CONFIG["hidden_size_4"], 
    output=CONFIG["output"]
)

Чтобы посмотреть из чего вообще состоит модель, можно воспользовать print.

In [ ]:
print(model)

Если вы используете уже предобученную модель из torch, то print тоже будет работать и покажет вам весь внутренний мир модели.

Дальше определяем функцию потерь и метод, по которому будет считаться градиент. 

In [ ]:
# функция потерь
loss=nn.CrossEntropyLoss()
# алгоритм для расчёта градиентного спуска 
optimizer=torch.optim.Adam(model.parameters(),lr=CONFIG["lr_rate"])

Это была подготовительная стадия. Теперь давайте организуем цикл для обучения.

In [ ]:
NUM_EPOCHS = 10

for epoch in range(NUM_EPOCHS):
    for i, (images, labels) in enumerate(train_dataloader):
        images = images.reshape(-1, 28 * 28)
        output=model(images)
        loss_value = loss(output, labels)
        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step() 
    
    print(f"Train. Epoch: {epoch}, loss={loss_value.item()}")  

In [ ]:
predicted=[]
with torch.no_grad():
    n_correct=0
    n_samples=0
    for images,labels in test_dataloader:
        images=images.reshape(-1,784)
        output=model(images) #applying the model we have built
        labels=labels
        _,prediction=torch.max(output,1)
        predicted.append(prediction)
print(prediction)

In [ ]:
for image, pred in zip(test_data, prediction):
    true_label = image[1]
    img = image[0].squeeze(0).cpu().numpy()

    plt.figure(figsize=[10, 8])
    plt.imshow(img)
    plt.title(f"true: {true_label} | Pred: {pred}")
    plt.show()

## Полезные ссылки 

* [MNIST Handwritten Digit Recognition Using Pytorch](https://medium.com/analytics-vidhya/training-mnist-handwritten-digit-data-using-pytorch-5513bf4614fb)